## Setup 

In [3]:
import torch
from torch.autograd import Variable as V
import torchvision.models as models
from torchvision import transforms as trn
from torch.nn import functional as F
import os
import numpy as np
from scipy.misc import imresize as imresize
from PIL import Image


def load_labels():
    # prepare all the labels
    # scene category relevant
    file_name_category = 'categories_places365.txt'
    if not os.access(file_name_category, os.W_OK):
        synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/categories_places365.txt'
        os.system('wget ' + synset_url)
    classes = list()
    with open(file_name_category) as class_file:
        for line in class_file:
            classes.append(line.strip().split(' ')[0][3:])
    classes = tuple(classes)

    # indoor and outdoor relevant
    file_name_IO = 'IO_places365.txt'
    if not os.access(file_name_IO, os.W_OK):
        synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/IO_places365.txt'
        os.system('wget ' + synset_url)
    with open(file_name_IO) as f:
        lines = f.readlines()
        labels_IO = []
        for line in lines:
            items = line.rstrip().split()
            labels_IO.append(int(items[-1]) -1) # 0 is indoor, 1 is outdoor
    labels_IO = np.array(labels_IO)

    # scene attribute relevant
    file_name_attribute = 'labels_sunattribute.txt'
    if not os.access(file_name_attribute, os.W_OK):
        synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/labels_sunattribute.txt'
        os.system('wget ' + synset_url)
    with open(file_name_attribute) as f:
        lines = f.readlines()
        labels_attribute = [item.rstrip() for item in lines]
    file_name_W = 'W_sceneattribute_wideresnet18.npy'
    if not os.access(file_name_W, os.W_OK):
        synset_url = 'http://places2.csail.mit.edu/models_places365/W_sceneattribute_wideresnet18.npy'
        os.system('wget ' + synset_url)
    W_attribute = np.load(file_name_W)

    return classes, labels_IO, labels_attribute, W_attribute

def hook_feature(module, input, output):
    features_blobs.append(np.squeeze(output.data.cpu().numpy()))

def returnCAM(feature_conv, weight_softmax, class_idx):
    # generate the class activation maps upsample to 256x256
    size_upsample = (256, 256)
    nc, h, w = feature_conv.shape
    output_cam = []
    for idx in class_idx:
        cam = weight_softmax[class_idx].dot(feature_conv.reshape((nc, h*w)))
        cam = cam.reshape(h, w)
        cam = cam - np.min(cam)
        cam_img = cam / np.max(cam)
        cam_img = np.uint8(255 * cam_img)
        output_cam.append(imresize(cam_img, size_upsample))
    return output_cam

def returnTF():
# load the image transformer
    tf = trn.Compose([
        trn.Resize((224,224)),
        trn.ToTensor(),
        trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    return tf


def load_model():
    # this model has a last conv feature map as 14x14

    model_file = 'wideresnet18_places365.pth.tar'
    if not os.access(model_file, os.W_OK):
        os.system('wget http://places2.csail.mit.edu/models_places365/' + model_file)
        os.system('wget https://raw.githubusercontent.com/csailvision/places365/master/wideresnet.py')

    import wideresnet
    model = wideresnet.resnet18(num_classes=365)
    checkpoint = torch.load(model_file, map_location=lambda storage, loc: storage)
    state_dict = {str.replace(k,'module.',''): v for k,v in checkpoint['state_dict'].items()}
    model.load_state_dict(state_dict)
    model.eval()



    # the following is deprecated, everything is migrated to python36

    ## if you encounter the UnicodeDecodeError when use python3 to load the model, add the following line will fix it. Thanks to @soravux
    #from functools import partial
    #import pickle
    #pickle.load = partial(pickle.load, encoding="latin1")
    #pickle.Unpickler = partial(pickle.Unpickler, encoding="latin1")
    #model = torch.load(model_file, map_location=lambda storage, loc: storage, pickle_module=pickle)

    model.eval()
    # hook the feature extractor
    features_names = ['layer4','avgpool'] # this is the last conv layer of the resnet
    for name in features_names:
        model._modules.get(name).register_forward_hook(hook_feature)
    return model

In [4]:
classes, labels_IO, labels_attribute, W_attribute = load_labels()
features_blobs = []
model = load_model()
tf = returnTF() # image transformer
params = list(model.parameters())
weight_softmax = params[-2].data.numpy()
weight_softmax[weight_softmax<0] = 0

## Split images into training and test sets

In [26]:
def split_images():
    """ 
    Sort image names alphabetically; every 5th image 
    will be a test image, and the rest train images.
    """
    path_to_images = '../data/images/'
    image_filenames = sorted(os.listdir(path_to_images))

    image_filenames[0], image_filenames[-1]
    train_filenames, test_filenames = [], []
    for (i, filename) in enumerate(image_filenames):
        
        if filename[-4:] != '.jpg':
            print("unexpected file: ", filename)
            continue
        
        if i % 5 == 0: test_filenames.append(filename)
        else: train_filenames.append(filename)

    print("Train images:", len(train_filenames))
    print('examples:', train_filenames[:3])
    print("Test images:", len(test_filenames))
    print('examples:', test_filenames[:3])

    return train_filenames, test_filenames

train_filenames, test_filenames = get_image_filenames()    

Train images: 13816
examples: ['w100909309_n333807633.jpg', 'w100909310_n40597297.jpg', 'w100913172_n4036977663.jpg']
Test images: 3454
examples: ['w100263755_n3861736779.jpg', 'w100927056_n2043277677.jpg', 'w101644823_n1368468414.jpg']


## Convert Images to Tensors

In [81]:
def image_to_tensor(filename):
    """ Given image filename, return a transformed tensor shape (1, C, H, W)"""
    img = Image.open(filename)
    input_img = tf(img).unsqueeze(0)
    return input_img

def get_final_tensor(filenames):
    """ Given a list of image filenames, return a tensor shape (N, C, H, W)"""
    tensors = tuple(image_to_tensor(filename) for filename in filenames)
    X = torch.cat(tensors, dim=0)
    print(X.shape)
    return X

In [ ]:
image_dir = "/Users/jsennett/Code/bike-friendly-street-classifier/data/images/"
X_test = get_final_tensor([image_dir + filename for filename in test_filenames])
X_train = get_final_tensor([image_dir + filename for filename in train_filenames])
print('X_train:', X_train.shape)
print('X_train:', X_test.shape)

In [ ]:
Y_test = torch.Tensor([labels.get(filename) for filename in test_filenames])
Y_train = torch.Tensor([labels.get(filename) for filename in train_filenames])

In [88]:
Y_train[:10]

tensor([0., 0., 1., 1., 0., 1., 0., 1., 1., 1.])

In [83]:
type(X_train)

torch.Tensor

In [ ]:
torch.cat(tuple((img_tensor, img_tensor)), dim=0)

In [59]:
len(test_filenames)

13816

In [ ]:
test_filename = train_filenames[0]
image_to_tensor(image_dir + test_filename)

In [51]:
img = Image.open(image_dir + test_filename)
img_tensor = tf(img).unsqueeze(0)

In [54]:
img2_tensor = torch.cat() img_tensor
img2_tensor.shape

torch.Size([1, 3, 224, 224])

In [31]:
import sys
sys.path.append("../scripts/")
from utils import get_image_labels

In [ ]:
image_labels = get_image_labels()

In [ ]:
from pprint import pprint 
pprint(image_labels[:10])

In [ ]:
labels = {row.get('filename'):row.get('label') for row in image_labels}

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [40]:
class FriendlyStreetDataset(Dataset):
    '''Friendly Street Dataset'''
    def __init__(self, csv_file, transform=None):
        """
        Args:
            image_dir: Path to image directory
            
            transform (callable): Optional transform to apply to sample
        """
        
        data = pd.read_csv(csv_file);
        self.X = np.array(data.iloc[:, 1:]).reshape(-1, 1, 28, 28)#.astype(float);
        self.Y = np.array(data.iloc[:, 0]);
        
        del data;
        self.transform = transform;
        
    def __len__(self):
        return len(self.X);
    
    def __getitem__(self, idx):
        item = self.X[idx];
        label = self.Y[idx];
        
        if self.transform:
            item = self.transform(item);
        
        return (item, label);


## Get an image

In [41]:
# load the test image
img_url = 'http://places.csail.mit.edu/demo/6.jpg'
os.system('wget %s -q -O test.jpg' % img_url)
img = Image.open('test.jpg')
input_img = V(tf(img).unsqueeze(0))

In [42]:
input_img.shape

torch.Size([1, 3, 224, 224])

In [43]:
dir_path = "/Users/jsennett/Code/bike-friendly-street-classifier/data/images/"
image_path = dir_path + "w302524881_n4100839885.jpg"

In [44]:
img = Image.open(image_path)

In [45]:
input_img = V(tf(img).unsqueeze(0))

In [46]:
input_img.shape

torch.Size([1, 3, 224, 224])